# A Distributed Matrix Data Structure and Its Statistical Applications on PyTorch

**NOTE: This notebook is incomplete. At this time, it is uploaded for testing purposes, in preparation of tutorials to be presented in the programming workshop at the inaugural Lange's Symposium on Feb 21-22, 2020.**

## Synopsis

We developed a distributed matrix operation package suitable for distributed matrix-vector operations and distributed tall-and-thin (or wide-and-short) matrices.
The code runs on both multi-node machines and multi-GPU machines using PyTorch.
We apply this package for four statistical applications, namely, nonnegative matrix factorization (NMF), multidimensional scaling (MDS), positron emission tomography (PET), and $\ell_1$-regularized Cox regression.
In particular, $\ell_1$-regularized Cox regression with the UK Biobank dataset was the biggest multivariate survival analysis to our knowledge. 
In this workshop, we provide small examples that run on a single node, and demonstrate multi-GPU usage on our own machine.

## Introduction to PyTorch

What is PyTorch?

## Basic PyTorch Operations

We introduce simple operations on PyTorch. Note that Python uses 0-based, rowmajor ordering, like C and C++ (R is 1-based, column-major ordering). First we import the PyTorch
library. This is equvalent to library() in R.

In [3]:
import torch

In [4]:
torch.__version__

'1.1.0'

### Tensor Creation

One may create an uninitialized tensor. This creates a 3 × 4 tensor (matrix).

In [5]:
torch.empty(3, 4) # uninitialized tensor

tensor([[1.8836e-27, 4.5705e-41, 1.8836e-27, 4.5705e-41],
        [1.4574e-43, 6.4460e-44, 1.4153e-43, 1.5274e-43],
        [1.5695e-43, 1.6255e-43, 1.6956e-43, 5.6052e-44]])

The following is equivalent to `set.seed()` in R.

In [6]:
torch.manual_seed(100)

This generates a tensor initialized with random values from (0, 1).

In [7]:
y = torch.rand(3, 4) # from Unif(0, 1)
y

tensor([[0.1117, 0.8158, 0.2626, 0.4839],
        [0.6765, 0.7539, 0.2627, 0.0428],
        [0.2080, 0.1180, 0.1217, 0.7356]])

We can also generate a tensor filled with zeros or ones.

In [8]:
z = torch.ones(3, 4) # torch.zeros(3, 4)
z

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

A tensor can be created from standard Python data.

In [9]:
w = torch.tensor([3, 4, 5, 6])
w

tensor([3, 4, 5, 6])

A tensor can be created in certain datatype (default: float32) and on certain device (default: CPU) of choice 

In [10]:
# double precision
w = torch.tensor([3, 4, 5, 6], dtype=torch.float64)
w

tensor([3., 4., 5., 6.], dtype=torch.float64)

In [11]:
# on GPU number zero. will not run if CUDA GPU is not present.
w = torch.tensor([3, 4, 5, 6], device='cuda:0')
w

tensor([3, 4, 5, 6], device='cuda:0')

Shape of a tensor can be accessed by appending `.shape` to the tensor name.

In [12]:
z.shape

torch.Size([3, 4])

### Casting

A tensor can change datatype and location by the method `.to()`. The arguments are similar to choosing datatype and device of the new tensor.

In [13]:
w = w.to(device = "cpu", dtype=torch.int32)
w

tensor([3, 4, 5, 6], dtype=torch.int32)

### Indexing

The following are standard method of indexing tensors.

In [14]:
y[2, 3] # indexing: zero-based, returns a 0-dimensional tensor

tensor(0.7356)

The indexing always returns a (sub)tensor, even for scalars (treated as zero-dimensional tensors).
A standard Python number can be returned by using .item().

In [15]:
y[2, 3].item() # A standard Python floating-point number

0.7355988621711731

To get a column from a tensor, we use the indexing as below. The syntax is similar but slightly
different from R.

In [16]:
y[:, 3] # 3rd column. The leftmost column is 0th. cf. y[, 4] in R

tensor([0.4839, 0.0428, 0.7356])

The following is for taking a row.

In [17]:
y[2, :] # 2nd row. The top row is 0th. cf. y[3, ] in R

tensor([0.2080, 0.1180, 0.1217, 0.7356])

### Simple operations

Here we provide an example of simple operations on PyTorch. Addition using the operator ‘+’ acts
just like anyone can expect:

In [18]:
x = y + z # a simple addition.
x

tensor([[1.1117, 1.8158, 1.2626, 1.4839],
        [1.6765, 1.7539, 1.2627, 1.0428],
        [1.2080, 1.1180, 1.1217, 1.7356]])

Here is another form of addition.

In [19]:
x = torch.add(y, z) # another syntax for addition

The operators ending with an underscore (`_`) changes the value of the tensor in-place. Otherwise, the argument never changes. Unlike methods ending with `!` in Julia, this rule is strictly enforced in PyTorch. (The underscore determines usage of the keyword `const` in C++-level.)

In [20]:
y.add_(z) # in-place addition

tensor([[1.1117, 1.8158, 1.2626, 1.4839],
        [1.6765, 1.7539, 1.2627, 1.0428],
        [1.2080, 1.1180, 1.1217, 1.7356]])

### Concatenation

We can concatenate the tensors using the function `cat()`, which resembles `c()`, `cbind()`, and
`rbind()` in R. The second argument indicates the dimension that the tesors are concatenated
along: zero means by concatenation by rows, and one means by columns.

In [21]:
torch.cat((y, z), 0) # along the rows

tensor([[1.1117, 1.8158, 1.2626, 1.4839],
        [1.6765, 1.7539, 1.2627, 1.0428],
        [1.2080, 1.1180, 1.1217, 1.7356],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000]])

In [22]:
torch.cat((y, z), 1) # along the columns

tensor([[1.1117, 1.8158, 1.2626, 1.4839, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.6765, 1.7539, 1.2627, 1.0428, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.2080, 1.1180, 1.1217, 1.7356, 1.0000, 1.0000, 1.0000, 1.0000]])

### Reshaping

One can reshape a tensor, like changing the attribute `dim` in R.

In [23]:
y.view(12) # 1-dimensional array

tensor([1.1117, 1.8158, 1.2626, 1.4839, 1.6765, 1.7539, 1.2627, 1.0428, 1.2080,
        1.1180, 1.1217, 1.7356])

Up to one of the arguments of `view()` can be −1. The size of the reshaped tensor is inferred
from the other dimensions.

In [24]:
# reshape into (6)-by-2 tensor;
# (6) is inferred from the other dimension
y.view(-1, 2)

tensor([[1.1117, 1.8158],
        [1.2626, 1.4839],
        [1.6765, 1.7539],
        [1.2627, 1.0428],
        [1.2080, 1.1180],
        [1.1217, 1.7356]])

### Basic statistics

Calling `.sum()`, `.mean()`, `.std()` methods of a tensor do the obvious. Optional argument determines the dimension of reduction.

In [25]:
y

tensor([[1.1117, 1.8158, 1.2626, 1.4839],
        [1.6765, 1.7539, 1.2627, 1.0428],
        [1.2080, 1.1180, 1.1217, 1.7356]])

In [26]:
y.sum()

tensor(16.5933)

In [27]:
y.sum(0) # reduces rows, columnwise sum

tensor([3.9962, 4.6878, 3.6469, 4.2623])

In [28]:
y.sum(1) # reduces columns, rowwise sum

tensor([5.6739, 5.7359, 5.1834])

In [29]:
y.sum((0, 1)) # reduces rows and columns -> a single number.

tensor(16.5933)

In [30]:
y.mean()

tensor(1.3828)

In [31]:
y.mean(0)

tensor([1.3321, 1.5626, 1.2156, 1.4208])

In [32]:
y.std(1)

tensor([0.3058, 0.3384, 0.2961])

### Linear Algebra

Matrix transpose is performed by appending `.t()` to a tensor. Matrix multiplication is carried out by the method `torch.mm()`.

In [33]:
torch.mm(y, z.t())

tensor([[5.6739, 5.6739, 5.6739],
        [5.7359, 5.7359, 5.7359],
        [5.1834, 5.1834, 5.1834]])

## `torch.distributed`: Distributed subpackage for PyTorch

`torch.distributed` is the subpackage for distributed operations on PyTorch. The interface is mostly inspired by the message passing interface (MPI). The available backends are:

* Gloo, a collective communication library developed by Facebook, included in PyTorch. Full support for CPU, partial collective communication only for GPU.
* MPI, a good-old communication standard. The most flexible, but PyTorch needs to be compiled from its source to use it as a backend. Full support for GPU if the MPI installation is "CUDA-aware".
* NCCL, Nvidia Collective Communications Library, collective communication only for multiple GPUs on the same machine.

For this workshop, we use Gloo for its full functionalities on CPU and runnability on Jupyter Notebook. The experiments in our paper use MPI for running multi-node setting and multi-GPU setting with basically the same code. The interface below is specific for Gloo backend. For MPI backend, please consult with a section from [distributed package tutorial](https://pytorch.org/tutorials/intermediate/dist_tuto.html#communication-backends) or [our code](https://github.com/kose-y/dist_stat/tree/master/examples).

In [34]:
import os
import torch
import torch.distributed as dist
from torch.multiprocessing import Process

def init_process(rank, size, fn, backend='gloo'):
    """ Initialize the distributed environment. """
    os.environ['MASTER_ADDR'] = '127.0.0.1'
    os.environ['MASTER_PORT'] = '29500'
    dist.init_process_group(backend, rank=rank, world_size=size)
    fn(rank, size)

def run_process(size, fn):
    processes = []
    for rank in range(size):
        p = Process(target=init_process, args=(rank, size, fn))
        p.start()
        processes.append(p)
        
    for p in processes:
        p.join()

### Point-to-point communication

![](https://pytorch.org/tutorials/_images/send_recv.png)
Figure courtesy of: https://pytorch.org/tutorials/_images/send_recv.png

In [35]:
"""Blocking point-to-point communication."""

def point_to_point(rank, size):
    tensor = torch.zeros(1)
    if rank == 0:
        tensor += 1
        # Send the tensor to process 1
        dist.send(tensor=tensor, dst=1)
    elif rank == 1:
        # Receive tensor from process 0
        dist.recv(tensor=tensor, src=0)
    dist.barrier()
    print('Rank ', rank, ' has data ', tensor[0])

In [36]:
run_process(4, point_to_point)

Rank  1  has data  tensor(1.)
Rank  0  has data  tensor(1.)
Rank  2  has data  tensor(0.)
Rank  3  has data  tensor(0.)


### Collective communication

| | | 
|:---|:---|
| ![](https://pytorch.org/tutorials/_images/scatter.png) | ![](https://pytorch.org/tutorials/_images/gather.png) |
| Scatter | Gather |
| ![](https://pytorch.org/tutorials/_images/reduce.png) | ![](https://pytorch.org/tutorials/_images/all_reduce.png) |
| Reduce | All-reduce |
| ![](https://pytorch.org/tutorials/_images/broadcast.png) | ![](https://pytorch.org/tutorials/_images/all_gather.png) |
| Broadcast | All-gather |

Table courtesy of: https://pytorch.org/tutorials/intermediate/dist_tuto.html#communication-backends


In [37]:
def broadcast(rank, size):
    tensor = torch.zeros(1)
    if rank == 0:
        tensor[0] = 7
    dist.broadcast(tensor, src=0)
    print('Rank ', rank, ' has data ', tensor[0])

In [38]:
run_process(4, broadcast)

Rank  0  has data  tensor(7.)
Rank  3  has data  tensor(7.)
Rank  2  has data  tensor(7.)
Rank  1  has data  tensor(7.)


In [39]:
def reduce(rank, size):
    tensor = torch.ones(1)
    dist.reduce(tensor, 3)
    print('Rank ', rank, ' has data ', tensor[0])

In [40]:
run_process(4, reduce)

Rank  3  has data  tensor(4.)
Rank  0  has data  tensor(4.)
Rank  1  has data  tensor(3.)
Rank  2  has data  tensor(2.)


In [41]:
def all_reduce(rank, size):
    tensor = torch.ones(1)
    dist.all_reduce(tensor, op=dist.ReduceOp.SUM)
    print('Rank ', rank, ' has data ', tensor[0])

In [42]:
run_process(4, all_reduce)

Rank  2  has data  tensor(4.)
Rank  1  has data  tensor(4.)
Rank  3  has data  tensor(4.)
Rank  0  has data  tensor(4.)


In [43]:
def all_gather(rank, size):
    tensors = [torch.zeros(1) for i in range(size)]
    dat = torch.zeros(1)
    dat[0] += rank
    dist.all_gather(tensors, dat)
    print('Rank ', rank, ' has data ', tensors)

In [44]:
run_process(4, all_gather)

Rank  2  has data  [tensor([0.]), tensor([1.]), tensor([2.]), tensor([3.])]
Rank  3  has data  [tensor([0.]), tensor([1.]), tensor([2.]), tensor([3.])]
Rank  1  has data  [tensor([0.]), tensor([1.]), tensor([2.]), tensor([3.])]
Rank  0  has data  [tensor([0.]), tensor([1.]), tensor([2.]), tensor([3.])]


## `distmat`: Distributed Matrices on PyTorch

Using the tensor operations and communication package, we created a data structure for a distributed matrix. In this structure, each process,
enumerated by its rank, holds a contiguous block of the full data matrix by rows or columns.
The data may be a sparse matrix. If GPUs are involved, each process controls a GPU whose
index matches the process rank. For notational simplicity, we denote the dimension to split
in square brackets. If a [100] × 100 matrix is split over four processes, the process with rank
0 keeps the first 25 rows of the matrix, and the rank 3 process takes the last 25 rows. For
the sake of simplicity, we always assume that the size along the split dimension is divided
by the number of processes.

In [45]:
import dist_stat.distmat as distmat

### Creation

- `distgen_ones()`: Creates a distributed matrix filled with ones
- `distgen_zeros()`: Creates a distributed matrix filled with zeros
- `distgen_uniform()`: Creates a distributed matrix from uniform distribution
- `distgen_normal()`: Creates a distributed matrix from stndard normal distribution 

In [46]:
def create_unif(rank, size):
    A = distmat.distgen_uniform(8, 8, TType=torch.DoubleTensor)
    print('Matrix A:', 'Rank ', rank, ' has data ', A.chunk)   

In [47]:
run_process(4, create_unif)

Matrix A: Rank  1  has data  tensor([[0.5459, 0.3573, 0.2406, 0.5600, 0.6501, 0.9692, 0.5168, 0.2422],
        [0.4312, 0.5917, 0.3425, 0.2202, 0.7030, 0.5629, 0.9259, 0.7612]],
       dtype=torch.float64)Matrix A: Rank  2  has data  tensor([[0.8381, 0.4943, 0.5984, 0.6167, 0.6128, 0.8593, 0.1344, 0.5146],
        [0.1479, 0.4238, 0.5144, 0.7051, 0.8133, 0.1795, 0.2721, 0.4631]],
       dtype=torch.float64)Matrix A: Rank  3  has data  tensor([[0.6214, 0.7096, 0.7099, 0.7977, 0.8779, 0.1236, 0.5650, 0.0655],
        [0.1210, 0.0454, 0.5070, 0.1860, 0.5035, 0.1454, 0.8090, 0.4991]],
       dtype=torch.float64)


Matrix A: Rank  0  has data  tensor([[0.6941, 0.3464, 0.9751, 0.7911, 0.4274, 0.4460, 0.5522, 0.9559],
        [0.9405, 0.2215, 0.3271, 0.1352, 0.6283, 0.3030, 0.1302, 0.1811]],
       dtype=torch.float64)


A distributed matrix can also be created from local chunks.

In [48]:
def from_chunks(rank, size):
    torch.manual_seed(100 + rank)
    chunk = torch.randn(2, 4)
    print("rank ", rank, "has chunk", chunk)
    A = distmat.THDistMat.from_chunks(chunk)
    print('Matrix A:', 'Rank ', rank, ' has data ', A.chunk)    
    if rank == 0:
        print(A.shape)

In [49]:
run_process(4, from_chunks)

rank  3 has chunk tensor([[ 1.7286, -0.4007,  2.5587,  1.6848],
        [-1.6571, -0.2811,  0.7743, -0.9554]])
rank  2 has chunk tensor([[ 0.9907,  0.3349,  1.1497, -0.5498],
        [-0.1046,  2.0104, -0.7886, -0.1246]])
rank  0 has chunk tensor([[ 0.3607, -0.2859, -0.3938,  0.2429],
        [-1.3833, -2.3134, -0.3172, -0.8660]])
rank  1 has chunk tensor([[-1.3905, -0.8152, -0.3204,  0.7377],
        [-1.7534,  0.6033, -0.2520, -0.4373]])
Matrix A: Rank  3  has data  tensor([[ 1.7286, -0.4007,  2.5587,  1.6848],
        [-1.6571, -0.2811,  0.7743, -0.9554]])Matrix A: Rank  1  has data  tensor([[-1.3905, -0.8152, -0.3204,  0.7377],
        [-1.7534,  0.6033, -0.2520, -0.4373]])Matrix A: Rank  2  has data  tensor([[ 0.9907,  0.3349,  1.1497, -0.5498],
        [-0.1046,  2.0104, -0.7886, -0.1246]])Matrix A: Rank  0  has data  tensor([[ 0.3607, -0.2859, -0.3938,  0.2429],
        [-1.3833, -2.3134, -0.3172, -0.8660]])



[8, 4]


Data can be distributed from the master process.

In [64]:
def dist_data(rank, size):
    if rank == 0:
        data = torch.rand(4, 2)
        print("master data: ", data)
    else:
        data = None
    
    data_dist = distmat.dist_data(data, src=0, TType=torch.DoubleTensor)
    print('data_dist: ', 'Rank ', rank, ' has data ', data_dist.chunk)

In [65]:
run_process(4, dist_data)

master data:  tensor([[0.7118, 0.7876],
        [0.4183, 0.9014],
        [0.9969, 0.7565],
        [0.2239, 0.3023]])
data_dist:  Rank  0  has data  tensor([[0.7118, 0.7876]], dtype=torch.float64)
data_dist:  Rank  2  has data  tensor([[0.9969, 0.7565]], dtype=torch.float64)
data_dist:  Rank  1  has data  tensor([[0.4183, 0.9014]], dtype=torch.float64)
data_dist:  Rank  3  has data  tensor([[0.2239, 0.3023]], dtype=torch.float64)


### Elementwise operations

Some of the basic functions work naturally. 

In [84]:
def elemwise_1(rank, size):
    A = distmat.distgen_uniform(4, 2)
    print("A: rank ", rank, "has chunk", A.chunk)
    B = distmat.distgen_uniform(4, 2)
    print("B: rank ", rank, "has chunk", B.chunk)    
    C = A + B
    print("C: rank ", rank, "has chunk", C.chunk)    
    C += A
    print("C: rank ", rank, "has chunk", C.chunk)
    
    logC = C.log()
    print("logC: rank ", rank, "has chunk", logC.chunk) 

In [85]:
run_process(4, elemwise_1)

A: rank  0 has chunk tensor([[0.6941, 0.3464]], dtype=torch.float64)
A: rank  3 has chunk tensor([[0.5522, 0.9559]], dtype=torch.float64)
A: rank  2 has chunk tensor([[0.4274, 0.4460]], dtype=torch.float64)
A: rank  1 has chunk tensor([[0.9751, 0.7911]], dtype=torch.float64)
B: rank  0 has chunk tensor([[0.9405, 0.2215]], dtype=torch.float64)
B: rank  3 has chunk tensor([[0.1302, 0.1811]], dtype=torch.float64)
B: rank  2 has chunk tensor([[0.6283, 0.3030]], dtype=torch.float64)
B: rank  1 has chunk tensor([[0.3271, 0.1352]], dtype=torch.float64)
C: rank  0 has chunk tensor([[1.6346, 0.5680]], dtype=torch.float64)
C: rank  1 has chunk tensor([[1.3022, 0.9263]], dtype=torch.float64)
C: rank  2 has chunk tensor([[1.0556, 0.7490]], dtype=torch.float64)
C: rank  3 has chunk tensor([[0.6824, 1.1369]], dtype=torch.float64)
C: rank  0 has chunk tensor([[2.3287, 0.9144]], dtype=torch.float64)
C: rank  1 has chunk tensor([[2.2773, 1.7175]], dtype=torch.float64)
C: rank  3 has chunk tensor([[1.23

Broadcasting (similar to Julia's dot broadcasting) also works as expected:

In [98]:
def dim_broadcasting(rank, size):
    A = distmat.distgen_uniform(4, 2) # [4] x 2 
    print("A: rank ", rank, "has chunk", A.chunk)
    B = distmat.distgen_ones(4, 1) # [4] x 1
    print("B: rank ", rank, "has chunk", B.chunk)
    A += B # B treated as [4] x 2 matrix
    print("A: rank ", rank, "has chunk", A.chunk)
    C = 2 * torch.ones(1, 2, dtype=torch.float64) # 1 x 2
    A += C # C treated as [4] x 2 matrix
    print("A: rank ", rank, "has chunk", A.chunk)     

In [99]:
run_process(4, dim_broadcasting)

A: rank  3 has chunk tensor([[0.5522, 0.9559]], dtype=torch.float64)
A: rank  1 has chunk tensor([[0.9751, 0.7911]], dtype=torch.float64)
A: rank  2 has chunk tensor([[0.4274, 0.4460]], dtype=torch.float64)
A: rank  0 has chunk tensor([[0.6941, 0.3464]], dtype=torch.float64)
B: rank  0 has chunk tensor([[1.]], dtype=torch.float64)
B: rank  3 has chunk tensor([[1.]], dtype=torch.float64)
B: rank  2 has chunk tensor([[1.]], dtype=torch.float64)
B: rank  1 has chunk tensor([[1.]], dtype=torch.float64)
A: rank  0 has chunk tensor([[1.6941, 1.3464]], dtype=torch.float64)
A: rank  1 has chunk tensor([[1.9751, 1.7911]], dtype=torch.float64)
A: rank  3 has chunk tensor([[1.5522, 1.9559]], dtype=torch.float64)
A: rank  2 has chunk tensor([[1.4274, 1.4460]], dtype=torch.float64)
A: rank  1 has chunk tensor([[3.9751, 3.7911]], dtype=torch.float64)
A: rank  0 has chunk tensor([[3.6941, 3.3464]], dtype=torch.float64)
A: rank  2 has chunk tensor([[3.4274, 3.4460]], dtype=torch.float64)
A: rank  3 ha

For general functions, we have `.apply()` and `.apply_binary()`.

In [100]:
def elemwise_2(rank, size):
    A = distmat.distgen_uniform(4, 2)
    print("A: rank ", rank, "has chunk", A.chunk)
    B = distmat.distgen_uniform(4, 2)
    print("B: rank ", rank, "has chunk", B.chunk) 
    Asqp1 = A.apply(lambda x: x**2 + 1)
    print("Asqp1: rank ", rank, "has chunk", Asqp1.chunk)    
    AsqpBsq = A.apply_binary(B, lambda x, y: x**2 + y**2)
    print("AsqpBsq: rank ", rank, "has chunk", AsqpBsq.chunk) 

In [101]:
run_process(4, elemwise_2)

A: rank  2 has chunk tensor([[0.4274, 0.4460]], dtype=torch.float64)
A: rank  0 has chunk tensor([[0.6941, 0.3464]], dtype=torch.float64)
A: rank  1 has chunk tensor([[0.9751, 0.7911]], dtype=torch.float64)
A: rank  3 has chunk tensor([[0.5522, 0.9559]], dtype=torch.float64)
B: rank  1 has chunk tensor([[0.3271, 0.1352]], dtype=torch.float64)
B: rank  2 has chunk tensor([[0.6283, 0.3030]], dtype=torch.float64)
B: rank  0 has chunk tensor([[0.9405, 0.2215]], dtype=torch.float64)
B: rank  3 has chunk tensor([[0.1302, 0.1811]], dtype=torch.float64)
Asqp1: rank  1 has chunk tensor([[1.9508, 1.6259]], dtype=torch.float64)
Asqp1: rank  0 has chunk tensor([[1.4818, 1.1200]], dtype=torch.float64)
Asqp1: rank  2 has chunk tensor([[1.1826, 1.1989]], dtype=torch.float64)
Asqp1: rank  3 has chunk tensor([[1.3049, 1.9137]], dtype=torch.float64)
AsqpBsq: rank  2 has chunk tensor([[0.5774, 0.2907]], dtype=torch.float64)
AsqpBsq: rank  3 has chunk tensor([[0.3219, 0.9465]], dtype=torch.float64)
AsqpBs

### Reductions (sum, max, min)

Summations, minimums, and maximums can be carried out in a way similar to local tensors.

In [74]:
def reductions(rank, size):
    A = distmat.distgen_uniform(4, 2)
    print("A: rank ", rank, "has chunk", A.chunk)
    print("sum of A: ", A.sum())
    print("maximum of A: ", A.max())
    print("minimum of A: ", A.min())
    
    sumA_row = A.sum(0) # row sum, a tensor with the same values on all processes 
    sumA_col = A.sum(1) # col sum, a distributed matrix
    print("row sum of A: ", sumA_row)
    
    print("sumA_col: rank ", rank, "has chunk", sumA_col.chunk)

In [75]:
run_process(4, reductions)

A: rank  1 has chunk tensor([[0.9751, 0.7911]], dtype=torch.float64)
A: rank  3 has chunk tensor([[0.5522, 0.9559]], dtype=torch.float64)
A: rank  2 has chunk tensor([[0.4274, 0.4460]], dtype=torch.float64)
A: rank  0 has chunk tensor([[0.6941, 0.3464]], dtype=torch.float64)


/DATA/home/kose/anaconda3/lib/python3.6/site-packages/torch/distributed/distributed_c10d.py:100: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "
/DATA/home/kose/anaconda3/lib/python3.6/site-packages/torch/distributed/distributed_c10d.py:100: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "
/DATA/home/kose/anaconda3/lib/python3.6/site-packages/torch/distributed/distributed_c10d.py:100: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "
/DATA/home/kose/anaconda3/lib/python3.6/site-packages/torch/distributed/distributed_c10d.py:100: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.Re

sum of A:  tensor(5.1882, dtype=torch.float64)
sum of A:  tensor(5.1882, dtype=torch.float64)
sum of A:  tensor(5.1882, dtype=torch.float64)
sum of A:  tensor(5.1882, dtype=torch.float64)
maximum of A:  tensor(0.9751, dtype=torch.float64)
maximum of A:  tensor(0.9751, dtype=torch.float64)
maximum of A:  tensor(0.9751, dtype=torch.float64)
maximum of A:  tensor(0.9751, dtype=torch.float64)
minimum of A:  tensor(0.3464, dtype=torch.float64)
minimum of A:  tensor(0.3464, dtype=torch.float64)
minimum of A:  tensor(0.3464, dtype=torch.float64)
minimum of A:  tensor(0.3464, dtype=torch.float64)
row sum of A:  tensor([[2.6488, 2.5394]], dtype=torch.float64)
row sum of A:  tensor([[2.6488, 2.5394]], dtype=torch.float64)
row sum of A:  tensor([[2.6488, 2.5394]], dtype=torch.float64)
row sum of A:  tensor([[2.6488, 2.5394]], dtype=torch.float64)
sumA_col: rank  2 has chunk tensor([[0.8733]], dtype=torch.float64)
sumA_col: rank  0 has chunk tensor([[1.0406]], dtype=torch.float64)
sumA_col: rank  

### Diagonals

In [82]:
def diagonals(rank, size):
    if rank == 0:
        p = 4
        data = torch.randn(p, p)
        print("master data: ", data)
    else:
        data = None
        
    data_dist = distmat.dist_data(data, src=0, TType=torch.DoubleTensor)
    
    diag1 = data_dist.diag() # distributed diagonal
    print("diag1: rank ", rank, "has chunk", diag1.chunk)
    
    diag2 = data_dist.diag(distribute=False) # diagonal gathered in each process
    print("diag2: ", diag2)
    
    data_dist.fill_diag_(0) # fill the diagonals with zeros
    print("data_dist: rank ", rank, "has chunk", data_dist.chunk)

In [83]:
run_process(4, diagonals)

master data:  tensor([[ 0.6857,  0.7877, -0.9778,  2.1302],
        [-3.1896,  1.5914, -0.0247, -0.8466],
        [ 1.4205, -1.5741, -0.3572, -0.3097],
        [ 1.1705, -0.5410, -0.7116,  0.0575]])
diag1: rank  0 has chunk tensor([[0.6857]], dtype=torch.float64)
diag1: rank  2 has chunk tensor([[-0.3572]], dtype=torch.float64)
diag1: rank  3 has chunk tensor([[0.0575]], dtype=torch.float64)
diag1: rank  1 has chunk tensor([[1.5914]], dtype=torch.float64)
diag2:  tensor([[ 0.6857],
        [ 1.5914],
        [-0.3572],
        [ 0.0575]], dtype=torch.float64)diag2:  tensor([[ 0.6857],
        [ 1.5914],
        [-0.3572],
        [ 0.0575]], dtype=torch.float64)diag2:  tensor([[ 0.6857],
        [ 1.5914],
        [-0.3572],
        [ 0.0575]], dtype=torch.float64)diag2:  tensor([[ 0.6857],
        [ 1.5914],
        [-0.3572],
        [ 0.0575]], dtype=torch.float64)



data_dist: rank  2 has chunk tensor([[ 1.4205, -1.5741,  0.0000, -0.3097]], dtype=torch.float64)
data_dist: rank  3 

### Matrix multiplications

Six different scenarios of matrix-matrix multiplications, each representing a different configuration of the split dimension of two input
matrices and the output matrix, were considered and implemented. 

< Content of Table 1 here >



The implementation of each case is carried
out using the collective communication directives. Matrix multiplication scenarios are automatically selected based on the shapes of the input matrices A and
B, except for the Scenarios 1 and 3 sharing the same input structure. Those two are further
distinguished by the shape of output, AB. The nonnegative matrix factorization involves Scenarios 1 to 5.
Scenario 6 is for matrix-vector multiplications, where broadcasting small vectors is almost
always efficient.

In [85]:
from dist_stat.distmm import *
def test_distmm(rank, size):
    TType = torch.DoubleTensor
    p = 12; q = 8; r = 2
    if rank==0:
        fat   = TType(p, q).normal_()
        thin1 = TType(q, r).normal_()
        thin2 = TType(p, r).normal_()

    else:
        fat, thin1, thin2 = None, TType(q,r), TType(p,r)
    dist.broadcast(thin1,0)
    dist.broadcast(thin2,0)

    fat_dist   = distmat.dist_data(fat, src=0, TType=TType)
    thin1_dist = distmat.dist_data(thin1, src=0, TType=TType)
    thin2_dist = distmat.dist_data(thin2, src=0, TType=TType)

    # test distmm_thinthin_inner 
    if rank==0: 
        print("distmm_thinthin_inner: thin1^T x thin1")
        thin1_thin1 = torch.mm(torch.t(thin1), thin1)
        print("thin1^T x thin1: ", thin1_thin1)

    thin1_thin1_bd = distmm_thinthin_inner(thin1_dist.t(), thin1_dist)
    print("rslt in rank %d: "%(rank,), thin1_thin1_bd)

    dist.barrier()


    if rank==0:
        print("distmm_db_d: thin2 x thin1_thin1")
        correct = torch.mm(thin2, thin1_thin1)
        print(correct)
    rslt_dist = distmm_db_d(thin2_dist, thin1_thin1_bd)
    print("rslt in rand %d: "%(rank,), rslt_dist.chunk)
    print(rslt_dist.byrow)


    dist.barrier()

    if rank==0:
        print("distmm_db_d (reverse): thin1_thin1 x thin2^T")
        correct = torch.mm(thin1_thin1, torch.t(thin2))
        print(correct)
    rslt_dist = distmm_db_d(thin2_dist.t(), thin1_thin1_bd, True)
    print("rslt in rand %d: "%(rank,), rslt_dist.chunk)
    print(rslt_dist.byrow)

    dist.barrier()

    if rank==0:
        print("_distmm_fatthin_byrow: fat x thin1")
        correct = torch.mm(fat, thin1)
        print(correct)
    rslt_dist = distmm_fatthin(fat_dist, thin1_dist)
    print("rslt in rand %d: "%(rank,), rslt_dist.chunk)
    print(rslt_dist.byrow)

    dist.barrier()

    if rank==0:
        print("_distmm_fatthin_byrow (reverse): thin1^T x fat^T")
        correct = torch.mm(torch.t(thin1), torch.t(fat))
        print(correct)
    rslt_dist = distmm_fatthin(fat_dist.t(), thin1_dist.t(), reverse=True)
    print("rslt in rand %d: "%(rank,), rslt_dist.chunk)
    print(rslt_dist.byrow)

    dist.barrier()

    if rank==0:
        print("_distmm_thinfat_byrow: thin2^T x fat" )
        # Note: this is reverse for distmm_fatthin, non-reverse for inner ftn
        correct = torch.mm(torch.t(thin2), fat)
        print(correct)
    rslt_dist = distmm_fatthin(fat_dist, thin2_dist.t(), reverse=True, 
                                out_sizes=thin1_dist.sizes)
    print("rslt in rand %d: "%(rank,), rslt_dist.chunk)
    print(rslt_dist.byrow)

    dist.barrier()

    if rank==0:
        print("_distmm_thinfat_byrow (reverse): fat^T x thin2" )
        # Note: this is reverse for distmm_fatthin, non-reverse for inner ftn
        correct = torch.mm(torch.t(fat), thin2)
        print(correct)
    rslt_dist = distmm_fatthin(fat_dist.t(), thin2_dist, reverse=False, 
                                out_sizes=thin1_dist.sizes)
    print("rslt in rand %d: "%(rank,), rslt_dist.chunk)
    print(rslt_dist.byrow)

    if rank==0:
        print("distmm_thinthin_outer: thin1 x thin2^T" )
        correct = torch.mm(thin1, torch.t(thin2))
        print(correct)
    rslt_dist = distmm_thinthin_outer(thin1_dist, thin2_dist.t())
    print("rslt in rank %d: "%(rank,), rslt_dist.chunk)
    print(rslt_dist.byrow)

    if rank==0:
        print("distmm_db_b: thin1^T x thin1(dense)" )
        correct = torch.mm(torch.t(thin1), thin1)
        print(correct)
    rslt_dist = distmm_db_b(thin1_dist.t(), thin1)
    print("rslt in rank %d: "%(rank,), rslt_dist)


    dist.barrier()
    if rank==0:
        print("now we check distributed sparse matrices.")

    def to_sparse(x):
        """ converts dense tensor x to sparse format """
        x_typename = torch.typename(x).split('.')[-1]
        sparse_tensortype = getattr(torch.sparse, x_typename)

        indices = torch.nonzero(x)
        if len(indices.shape) == 0:  # if all elements are zeros
            return sparse_tensortype(*x.shape)
        indices = indices.t()
        values = x[tuple(indices[i] for i in range(indices.shape[0]))]
        return sparse_tensortype(indices, values, x.size())
    
    thin1_sparse_chunk = to_sparse(thin1_dist.chunk)
    thin1_sparse_dist = THDistMat.from_chunks(thin1_sparse_chunk)
    thin2_sparse_chunk = to_sparse(thin2_dist.chunk)
    thin2_sparse_dist = THDistMat.from_chunks(thin2_sparse_chunk)
    print(thin1_sparse_chunk.shape)
    print(thin1_sparse_dist.t().shape)
    print(thin1_dist.shape)
    if rank==0:
        print("correct: ", torch.mm(thin1.t(), thin1))

    r =  distmat.mm(thin1_sparse_dist.t(), thin1_dist )
    print("rslt: ", r)
    

In [86]:
run_process(4, test_distmm)

distmm_thinthin_inner: thin1^T x thin1
thin1^T x thin1:  tensor([[18.8603, -5.5108],
        [-5.5108, 13.0393]], dtype=torch.float64)


/srv/conda/envs/notebook/lib/python3.7/site-packages/torch/distributed/distributed_c10d.py:100: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "
/srv/conda/envs/notebook/lib/python3.7/site-packages/torch/distributed/distributed_c10d.py:100: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "
/srv/conda/envs/notebook/lib/python3.7/site-packages/torch/distributed/distributed_c10d.py:100: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "
/srv/conda/envs/notebook/lib/python3.7/site-packages/torch/distributed/distributed_c10d.py:100: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.Reduce

rslt in rank 0:  tensor([[18.8603, -5.5108],
        [-5.5108, 13.0393]], dtype=torch.float64)
rslt in rank 1:  tensor([[18.8603, -5.5108],
        [-5.5108, 13.0393]], dtype=torch.float64)rslt in rank 2:  tensor([[18.8603, -5.5108],
        [-5.5108, 13.0393]], dtype=torch.float64)rslt in rank 3:  tensor([[18.8603, -5.5108],
        [-5.5108, 13.0393]], dtype=torch.float64)


distmm_db_d: thin2 x thin1_thin1
tensor([[ 20.4717, -20.9301],
        [ -0.3866,  -2.3102],
        [  3.4683,  -1.9985],
        [ 10.0434, -21.7488],
        [ -8.9008,   1.7786],
        [ 14.3755, -13.4916],
        [ 14.6289,   6.1289],
        [-11.1466,  -3.2908],
        [ -4.6314,  17.0375],
        [ -4.6952,  -2.8811],
        [ 27.1388,  -5.2625],
        [ -7.5063,  -3.7725]], dtype=torch.float64)
rslt in rand 3:  tensor([[-4.6952, -2.8811],
        [27.1388, -5.2625],
        [-7.5063, -3.7725]], dtype=torch.float64)rslt in rand 0:  tensor([[ 20.4717, -20.9301],
        [ -0.3866,  -2.3102],
      

## Nonnegative Matrix Factorization (NMF)

The following code is a simplified version. The full object-oriented version is available at the [GitHub repo](https://github.com/kose-y/dist_stat).

In [ ]:
def nmf(rank, size):
    # TODO: initialization here
    for i in range(maxiter):
        XWt =  distmat.mm(data, W.t())
        WWt =  distmat.mm(W, W.t())
        VWWt = distmat.mm(V, WWt)
        V.mul_(XWt).div_(VWWt)

        VtX  = distmat.mm(V.t(), data, out_sizes=W.sizes)
        VtV  = distmat.mm(V.t(), V)
        VtVW = distmat.mm(VtV, W)
        W = W.mul_(VtX).div_(VtVW)
        if i % 10 == 0:
            # print obj
            pass

## $\ell_1$-regularized Cox Regression

In [ ]:
def cox_l1(rank, size):
    # TODO: initialization here
    lambd = 0.01
    soft_threshold = torch.nn.Softshrink(lambd)
    for i in range(maxiter):
        Xbeta = distmat.mm(data, beta)
        w = Xbeta.exp()
        W = w.cumsum(0)
        w_dist = distmat.dist_data(w, TType=w.TType)
        pi = (w_dist/W.t()) * pi_ind
        pd  = distmat.mm(pi, delta)
        dmpd = delta_dist - pd
        grad = distmat.mm(datat, dmpd)
        beta = (beta + grad * sigma).apply(soft_threshold)
        if i % 10 == 0:
            # print obj
            pass

## Multi-GPU Demonstration

We demonstrate 10,000 x 10,000 examples on 2-8 GPUs on our server.

## Multi-node

The data structure can also be utilized on multi-node clusters. The structure was used for the analysis of 200,000 x 500,000 UK Biobank data.

## Future Direction

MPI-only, lightweight, more flexible version in Julia is in preparation. CUDA-aware MPI support for the central MPI interface [MPI.jl](https://github.com/JuliaParallel/MPI.jl) was added in the process.